# **Preprocessing these datasets in the correct formats is too time and resources consuming to do it on every execution, so we export the final dataframes to csv to reuse them directly elsewhere**

In [ ]:
from google.colab import drive#On monte le drive pour charger les datasets depuis google drive (optionnel)
drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive/

Mounted at /content/gdrive/
[Errno 107] Transport endpoint is not connected: 'gdrive/MyDrive/'
/content/gdrive/MyDrive


In [ ]:
import pandas as pd

X = pd.read_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/InputTrain.csv").iloc[:,2:]
Y1 = pd.read_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/StepTwo_LabelTrain_WashingMachine.csv").iloc[:,2:]
Y2 = pd.read_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/StepTwo_LabelTrain_Dishwasher.csv").iloc[:,2:]
Y3 = pd.read_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/StepTwo_LabelTrain_TumbleDryer.csv").iloc[:,2:]
Y4 = pd.read_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/StepTwo_LabelTrain_Microwave.csv").iloc[:,2:]
Y5 = pd.read_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/StepTwo_LabelTrain_Kettle.csv").iloc[:,2:]
dfXTest = pd.read_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/InputTest.csv").iloc[:,2:]


In [ ]:
from sklearn.preprocessing import StandardScaler
consumption_values = dfXTest.values
consumption_values = consumption_values.reshape(consumption_values.shape[0], -1)

# Remove the mean and scale the values to have unit variance
scaler = StandardScaler()
normalized_consumption_values = scaler.fit_transform(consumption_values)
dfXTest.iloc[:, 0:] = normalized_consumption_values.reshape(normalized_consumption_values.shape[0], -1)


In [ ]:
from sklearn.preprocessing import StandardScaler
consumption_values = X.values
consumption_values = consumption_values.reshape(consumption_values.shape[0], -1)

# Remove the mean and scale the values to have unit variance
scaler = StandardScaler()
normalized_consumption_values = scaler.fit_transform(consumption_values)
X.iloc[:, 0:] = normalized_consumption_values.reshape(normalized_consumption_values.shape[0], -1)


In [ ]:
y1 = Y1.melt().iloc[:,1]
y2 = Y2.melt().iloc[:,1]
y3 = Y3.melt().iloc[:,1]
y4 = Y4.melt().iloc[:,1]
y5 = Y5.melt().iloc[:,1]

y= pd.DataFrame()
y["Washing Machine"] = y1
y["Dishwasher"] = y2
y["Tumble Dryer"] = y3
y["Microwave"] = y4
y["Kettle"] = y5



import re
def extract_numbers(text):
    numbers = re.findall(r'\d+', text)
    return int(''.join(numbers))

X = X.melt()
X['variable'] = X['variable'].apply(extract_numbers)
X = X.rename(columns={'variable': 'Serie_number'})
lag_window = 3

for i in range(1, lag_window+1):
    X[f'lag_{i}'] = X['value'].shift(i)

X['rolling_mean-3'] = X['value'].rolling(window=3).mean()
X['rolling_mean-6'] = X['value'].rolling(window=6).mean()
X['rolling_mean-6'] = X['value'].rolling(window=9).mean()



# Drop rows with NaN values
X.dropna(inplace=True)
y = y.loc[X.index]

dfXTest = dfXTest.melt()
dfXTest['variable'] = dfXTest['variable'].apply(extract_numbers)
dfXTest = dfXTest.rename(columns={'variable': 'Serie_number'})



In [ ]:
X.to_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/Melted_Train2.csv", index=False)
y.to_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/Melted_Fused_Labels3.csv", index=False)
dfXTest.to_csv("/content/gdrive/MyDrive/M1 AI/Data Science/Project/step2/Melted_Test2.csv", index=False)

In [ ]:
# import xgboost as xgb
# from sklearn.metrics import average_precision_score
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4,random_state=42)

# dtrain = xgb.DMatrix(X_train,label=y_train)
# dtest = xgb.DMatrix(X_test,label=y_test)

# model = xgb.XGBModel()
# params = {
#     'max_depth': 10,
#     'eta': 0.1,
#     'objective': 'binary:logistic',
#     'eval_metric': 'error',
#     'booster':'dart',
#     'sample_type':'weighted',
#     'normalize_type':'forest',
#     'learning_rate':0.4,
#     'tree_method':'gpu_hist',
#     'n_jobs':500,
#     'rate_drop':0.3,
#     'skip_drop':0.2,

# }

# # Train model
# evalist = [(dtrain, 'train'), (dtest, 'eval')]
# model = xgb.train(params, dtrain,num_boost_round=70,verbose_eval=1)

# preds = model.predict(dtest) #Predictions
# y_test = y_test.astype(float)
# print("Precision = {}".format(average_precision_score(y_test.values, preds, average='macro')))
# print("Precision = {}".format(average_precision_score(y_test.values, preds, average='weighted')))


# #Precision = 0.25674527434942446 YANIS
# #Precision = 0.4107742871167801 YANIS


In [ ]:
# lag_window = 3

# for i in range(1, lag_window+1):
#     dfXTest[f'lag_{i}'] = dfXTest['value'].shift(i)

# dfXTest['rolling_mean-3'] = dfXTest['value'].rolling(window=3).mean()
# dfXTest['rolling_mean-6'] = dfXTest['value'].rolling(window=6).mean()


# dFin = xgb.DMatrix(dfXTest)

# preds_f = model.predict(dFin)

# dfF = pd.DataFrame(preds_f, columns=["Washing Machine","Dishwasher","Tumble Dryer","Microwave","Kettle"])
# dfF = dfF.rename_axis('Index')
# dfF.to_csv("no_features_baseline.csv")